In [108]:
with open("input15.txt") as f:
    inp = [x[:-1] for x in f.readlines()]
data = [[int(x) for x in y] for y in inp]

# Part 1

In [118]:
import numpy as np
from time import time

def dijkstra(data):
    nrows = len(data)
    ncols = len(data[0])
    #Initial node
    node = (0,0)
    #Set up sets of univisited nodes and visited nodes
    unvisited = {(i,j) for i in range(nrows) for j in range(ncols)}
    visited = set()
    #Set up initial tentative distance matrix
    tent_dist = [[np.inf for j in range(ncols)] for i in range(nrows)]
    tent_dist[0][0] = 0
    
    while len(unvisited) > 0:
        nbhs = [(node[0] + i, node[1] + j) for i in [-1,0,1] for j in [-1,0,1]\
                if (i==0 or j==0) and not(i==0 and j==0) and 0<=node[0]+i<=nrows-1 and 0<=node[1]+j<=ncols-1]
        # Compute new tentative distances for all unvisited nbhs of current node
        for pos in [x for x in nbhs if x not in visited]:
            new_dist = tent_dist[node[0]][node[1]] + data[pos[0]][pos[1]]
            if new_dist < tent_dist[pos[0]][pos[1]]:
                tent_dist[pos[0]][pos[1]] = new_dist
        # Remove current node from unvisited and add to visited
        visited.add(node)
        unvisited = unvisited - {node}
        # Select new node as the least tent distance node from unvisited
        if unvisited:
            node = min(unvisited, key=lambda x: tent_dist[x[0]][x[1]])
    return tent_dist[nrows-1][ncols-1]

def manhattan(p,q):
    return abs(p[0]-q[0])+abs(p[1]-q[1])

def a_star(data, h):
    # data is the cost matrix, h is the heuristic function
    nrows, ncols = len(data), len(data[0])
    # Define start and goal nodes
    start, goal = (0,0), (nrows-1, ncols-1)
    # Set up matrix of tentative distance from start
    gscore = [[np.inf for j in range(ncols)] for i in range(nrows)]
    gscore[0][0] = 0
    # Set up matrix of tentative distance to goal following heuristic h 
    fscore = [[np.inf for j in range(ncols)] for i in range(nrows)]
    fscore[0][0] = h(start, goal)
    # Set up the open set
    open_set = {start}
    
    while len(open_set) > 0:
        # Set current node to the one having min tent dist to goal
        node = min(open_set, key=lambda x: fscore[x[0]][x[1]])
        open_set = open_set - {node}
        # Define nbhs of current node
        nbhs = [(node[0] + i, node[1] + j) for i in [-1,0,1] for j in [-1,0,1]\
                if (i==0 or j==0) and not(i==0 and j==0) and 0<=node[0]+i<=nrows-1 and 0<=node[1]+j<=ncols-1]

        for pos in nbhs:
            tent_gscore = gscore[node[0]][node[1]] + data[pos[0]][pos[1]] 
            if tent_gscore < gscore[pos[0]][pos[1]]:
                gscore[pos[0]][pos[1]] = tent_gscore
                fscore[pos[0]][pos[1]] = tent_gscore + h(pos, goal)
                if pos not in open_set:
                    open_set.add(pos)
        
    return fscore[goal[0]][goal[1]]

# Dijkstra
start = time()
print(dijkstra(data))
end=time()
print(end-start)

#A*
start = time()
print(a_star(data, manhattan))
end=time()
print(end-start)

811
6.4087207317352295
811
0.18963885307312012


# Part 2

In [113]:
def new_tile(data, k):
    """New tile with k risk added"""
    nrows, ncols = len(data), len(data[0])
    return [[data[i][j]+k if data[i][j]+k<10 else (data[i][j]+k)%10+1 for j in range(ncols)] for i in range(nrows)]

def repeat_map(data, n):
    new_data = []
    for i in range(n):
        new_data = new_data + new_tile(data, i)
    full_data = new_data.copy()
    for j in range(1,n):
        full_data = [full_data[i] + new_tile(new_data,j)[i] for i in range(len(new_data))]
    return full_data
    
full_map = repeat_map(data,5)
start = time()
print(a_star(full_map, manhattan))
end=time()
end-start

3012


27.561350345611572